# Engenharia e seleção de features

In [ ]:
import os
import pickle
import random
from functools import partial

import joblib  # Para salvar o modelo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import utils
from scikeras.wrappers import KerasRegressor
from scipy.stats import randint, uniform  # Para a distribuição de parâmetros

# Importa e configura o Scikit-learn para rotear metadados
from sklearn import set_config
from sklearn.linear_model import LassoCV
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.model_selection import GroupKFold, HalvingRandomSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

set_config(
    enable_metadata_routing=True
)  # Para garantir o uso do GroupKFold no modelo linear

# Observações:

1 - As bibliotecas que serão usadas neste projeto possuem fontes múltiplas de aleatoriedade. Por isso ressalta-se os seguintes pontos abaixo:

- numpy, random (do Python), e tensorflow usam geradores diferentes.

    - Fixar random_state em HalvingRandomSearchCV só controla o sorteio dos hiperparâmetros, não o comportamento interno da rede.

- TensorFlow e paralelismo introduzem aleatóriedade.

    - Por padrão, TensorFlow usa múltiplas threads e kernels otimizados (como cuDNN no GPU), que executam operações não determinísticas (principalmente Dropout e Dense).

- O KerasRegressor recria o modelo a cada chamada

    - Mesmo que o random_state do scikit-learn esteja fixo, cada vez que fit() é chamado, o Sequential() do TensorFlow usa um estado de aleatoriedade independente (a menos que se fixe isso manualmente dentro da função que cria o modelo).

Dadas as observações acima, fixa-se abaixo o parâmetro SEED para alguns métodos que serão utilizados neste notebook.



2 - Ao rodar o processo de criação do modelo, na minha máquina, com o parâmetro n_jobs diferente de 1, havia o seguinte warning:

   
    UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak. 
   

- Ao parar, todo o processo de seleção de parâmetros ótimos era comprometido e não havia reproducibilidade do modelo à cada rodada.

- Colocando o n_jobs = 1, garante-se a reproducibilidade do modelo, ao custo de aumentar bastante o tempo de treinamento do modelo

- Caso esse modelo seja rodado em uma máquina mais robusta, esse problema pode não ocorrer, sendo possível acelerar o tempo de treinamento ao mudar-se o valor do parâmetro n_jobs.

# Parâmetros

In [ ]:
# Desativar GPU (garante total determinismo)
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Fixar sementes globais
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Garante execução determinística, desativando otimizações não reprodutíveis
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"


n_jobs = 1

# Caso queira-se reaproveitar o cálculo (demorado) dos valores do shap e investigar apenas a explicabilidade do modelo
existing_model_shap = False

caminho_features_selecionadas = "../data/08_reporting/selected_linear_features.npy"

caminho_dados = "../data/02_intermediate/training_b_df.csv"

caminho_modelo = "../data/06_models/modelo_notebook_4.joblib"

caminho_shap = "../data/06_reporting/shap_values_calculados_linear_features.npy"

caminho_X_train_linear_features = "../data/03_primary/X_train_linear_features.npy"

caminho_y_train_linear_features = "../data/03_primary/y_train_linear_features.npy"

caminho_X_test_linear_features = (
    "../data/03_primary/X_test_final_para_shap_linear_features.npy"
)

caminho_y_test_linear_features = (
    "../data/03_primary/y_test_final_para_shap_linear_features.npy"
)

caminho_explainer = "../data/08_reporting/explainer_expected_value_linear_features.pkl"

# 1 - Lendo os dados

In [ ]:
df = pd.read_csv("../data/02_intermediate/training_b_df.csv")

In [ ]:
X_train = df.drop(columns=["r"])
y_train = df["r"]

In [ ]:
X_train

In [ ]:
y_train

# 2 - Engenharia de features

Ao conversar com a astrofísica Camila Novaes que forneceu os dados simulados estudados aqui, foi sugerida a criação da média ponderada abaixo:

In [ ]:
X_train = utils.cria_feature_media_ponderada(X_train)

print("Primeiras 5 linhas com a nova feature:")
print(X_train[["l_2", "l_3", "l_4", "l_511", "media_ponderada_Cls"]].head())

In [ ]:
X_train["media_ponderada_Cls"].isnull().sum()

In [ ]:
X_train["media_ponderada_Cls"].tail()

# 3 - Seleção de features

Antes de seguir com a seleção de feature, verifica-se abaixo o mapa de calor que mede o grau de correlação linear entre as features e o target.

In [ ]:
# 1. Junta-se as features de treino (X_train) e o target (y_train)
df_train_corr = pd.concat([X_train, y_train.rename("r")], axis=1)

# 2. Calcula-se a matriz de correlação (entre todas as features e o target)
corr_matrix = df_train_corr.corr()

# 3. Foca-se apenas na correlação com o Target
target_corr = corr_matrix["r"].sort_values(ascending=False)

# Opcional: Pega-se apenas as 20 features mais correlacionadas para melhor visualização
top_n = 20
top_corr_features = target_corr.head(
    top_n + 1
).index.tolist()  # +1 para incluir o próprio Target

# 4. Cria o mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix.loc[
        top_corr_features, top_corr_features
    ],  # Matriz reduzida para as top N
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    cbar=True,
)
plt.title(f"Mapa de Calor das {top_n} Features Mais Correlacionadas com o Target")
plt.show()

# 5. Imprimir os valores de correlação com o Target (para confirmação)
print("\nCorrelação de Pearson com o Target:")
print(target_corr.drop("r").head(10))  # Mostra as 10 mais correlacionadas

print("\nCorrelação de Pearson com o Target:")
print(target_corr.drop("r").tail(10))  # Mostra as 10 mais correlacionadas

In [ ]:
print("\nCorrelação de Pearson com o Target:")
print(target_corr.drop("r").tail(10))  # Mostra as 10 mais correlacionadas

## Observações 

- Coerência entre Dados e Modelo: O mapa de calor confirma um alto grau de correlação linear de Pearson entre o target e um subconjunto das features. Esta descoberta é coerente com a análise de explicabilidade anterior, onde os gráficos de dependência SHAP indicaram que o modelo aprendeu predominantemente relações quase lineares com essas features.

- Contexto Físico e Escalas Angulares Críticas (l<100): As features que demonstram a correlação linear mais forte estão notavelmente associadas a múltiplos angulares muito baixos (l<100). Esta faixa de l corresponde às maiores escalas angulares observáveis da CMB, onde o sinal é mais uniforme e menos sujeito a perturbações de pequena escala.


- Validação da Simulação: Este resultado também confirma que, apesar dos dados proverem de simulações, as relações encontradas corroboram com a física esperada tanto para as grandes escalas angulares quanto para pequenas escalas angulares da CMB, validando a representatividade dos dados utilizados no modelo.

# DUVIDA - PERGUNTAR PARA A CAMILA

Para os Cls com valor grande estamos olhando em escalas pequenas, por isso espera-se uma relação não linear entre target e essas features?

Pas os Cls com valor pequeno tem-se regiões mais largas de observação no céu, onde a CMB é mais homogenea e isotropica, seria isso a causa da linearidade?


# Seleção de features usando um modelo linear (LASSO)

In [ ]:
groups = y_train.to_numpy().ravel()

In [ ]:
# 1. Escalar os dados
scaler = StandardScaler()
X_train_scaled_lasso = scaler.fit_transform(X_train)
X_train_scaled_lasso = pd.DataFrame(X_train_scaled_lasso, columns=X_train.columns)

# 2. Configurar o GroupKFold
gkf = GroupKFold(n_splits=5)

# 3. Encontrar o Alpha ideal (Melhor parâmetro de regularização)
lasso_cv = LassoCV(cv=gkf, random_state=42, n_jobs=-1, max_iter=10000)

# 4. Ajustar o modelo com os grupos para evitar vazamento
# Agora, o LassoCV saberá como rotear 'groups' para o GroupKFold.
lasso_cv.fit(X_train_scaled_lasso, y_train.to_numpy().ravel(), groups=groups)
print(
    f"\nMelhor Alpha (Lambda) Encontrado pelo LASSO (com GroupKFold): {lasso_cv.alpha_:.5f}"
)

## Observação:

- No contexto do LASSO (e na biblioteca Scikit-learn), Alpha (α) é o nome dado ao parâmetro de regularização (também chamado de Lambda, λ em estatística).

- O α controla a "força da penalidade" aplicada aos coeficientes do modelo.

- Um α alto (α→∞) força mais coeficientes de features a se tornarem exatamente zero. Isso cria um modelo mais simples (com menos features), mas pode causar underfitting (subajuste).

- Um α baixo (α→0) torna o modelo muito parecido com a Regressão Linear padrão (OLS), com pouca ou nenhuma penalidade. Isso usa todas as features, mas aumenta o risco de overfitting (sobreajuste).

In [ ]:
# Mapear coeficientes
coefficients = pd.DataFrame({"Feature": X_train.columns, "Coefficient": lasso_cv.coef_})

# Filtrar apenas as features que NÃO foram zeradas (selecionadas)
selected_coefficients = coefficients[coefficients["Coefficient"] != 0].copy()

# Calcular o impacto absoluto para ordenação
selected_coefficients["Abs_Impact"] = np.abs(selected_coefficients["Coefficient"])

# Ordenar por impacto absoluto (do mais importante para o menos importante)
selected_coefficients = selected_coefficients.sort_values(
    by="Abs_Impact", ascending=False
).reset_index(drop=True)


# ----------------------------------------------------
# 1. VISUALIZAÇÃO DAS 10 FEATURES MAIS IMPORTANTES
# ----------------------------------------------------

TOP_N = 20
top_N_features = selected_coefficients.head(TOP_N)

print(f"\n--- TOP {TOP_N} Features Mais Importantes (LASSO Coeficientes) ---")
print("-" * 75)

# Usamos o método .style.format() do Pandas para formatar a saída com duas casas decimais,
# garantindo que apenas as colunas de coeficiente sejam formatadas.
styled_output = top_N_features.style.format(
    {
        "Coefficient": "{:.4f}",  # Formato para o valor do coeficiente
        "Abs_Impact": "{:.4f}",  # Formato para o impacto absoluto
    }
).hide(axis="index")  # Oculta o índice do DataFrame para uma tabela mais limpa

print(styled_output.to_string())


plt.figure(figsize=(12, 6))
sns.barplot(
    x="Abs_Impact",
    y="Feature",
    data=top_N_features,
    hue="Feature",  # Variável 'y' mapeada para 'hue'
    palette="viridis",
    legend=False,  # Desativa a legenda que seria criada por 'hue'
)
plt.title(f"Top {TOP_N} Importância de Features (Coeficientes LASSO)", fontsize=14)
plt.xlabel("Impacto Absoluto (|Coeficiente|)")
plt.ylabel("Feature")
plt.show()

In [ ]:
# 1. Obter os coeficientes do modelo
coeficientes = lasso_cv.coef_

# 2. Obter os nomes das features (certifique-se de que estão na ordem correta)
# Exemplo: Se X é um DataFrame
nomes_features = X_train.columns

# 3. Criar uma máscara booleana para coeficientes não-zero
features_selecionadas_mask = coeficientes != 0

# 4. Aplicar a máscara para obter os nomes das features
selected_feature_names = nomes_features[features_selecionadas_mask]

print("Total de Features Originais:", len(nomes_features))
print("Total de Features Selecionadas pelo LASSO:", len(selected_feature_names))
print("\nFeatures Selecionadas:\n", selected_feature_names.tolist())

## Observação

Para plotar os gráficos SHAP com a explicabilidade dos modelos, salva-se abaixo as features usadas na modelagem.

In [ ]:
nomes_features

In [ ]:
with open(caminho_features_selecionadas, "wb") as f:
    pickle.dump({"list": selected_feature_names}, f)

In [ ]:
# Checa se o arquivo foi salvo corretamente
with open(caminho_features_selecionadas, "rb") as f:
    loaded_data = pickle.load(f)

In [ ]:
loaded_data["list"]

## Observação

- Grande parte das features eliminadas pelo modelo linear foram para os valores de 'l' com índice alto, especialmente maiores que 400.

- Isso era esperado pela física ser altamente não-linear em pequenas escalas.

In [ ]:
X_train = X_train[selected_feature_names]

In [ ]:
X_train

## 4 - Normalizando os dados

In [ ]:
# 1. Escalonamento das Features (X)
X_scaler = MinMaxScaler()
X_train_scaled = X_scaler.fit_transform(X_train)

# 2. Escalonamento do Target (Y), se for Regressão Contínua
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_train_2d = y_train.values.reshape(-1, 1)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train_2d)

In [ ]:
X_train_scaled

In [ ]:
X_train_scaled.max()

In [ ]:
X_train_scaled.min()

In [ ]:
y_train_scaled.max()

In [ ]:
y_train_scaled.min()

## Salvando valores normalizados

As features normalizadas serão necessárias no notebook de explicabilidade dos modelos, por isso salva-se abaixo:




In [ ]:
np.save(caminho_X_train_linear_features, X_train_scaled)

np.save(caminho_y_train_linear_features, y_train_scaled)

# 4 - Construindo um modelo de redes neurais

In [ ]:
# Numero de neuronios de entrada na rede neural
neuronios_entrada = len(selected_feature_names)

# Cria uma função que já “fixa” neuronios_entrada
modelo_com_input = partial(
    utils.criar_modelo_regularizado, neuronios_entrada=neuronios_entrada
)

# 1. Distribuição de Parâmetros a serem testados (ranges ao invés de listas fixas)
param_distributions = {
    # Neurônios: número inteiro aleatório entre 32 e 256
    "model__neurons": randint(low=32, high=256),
    # Taxa de Aprendizado: valor contínuo aleatório em escala logarítmica
    # Ex: entre 1e-4 e 1e-2 (0.0001 e 0.01)
    "model__learning_rate": uniform(loc=0.0001, scale=0.0099),
    # Batch Size: valores discretos
    "batch_size": [16, 32, 64],
    # O dropout_rate ajuda a diminuir o overfitting
    "model__dropout_rate": uniform(loc=0.1, scale=0.4),  # Testar entre 10% e 50%
    # Epochs: valores discretos (o HRS vai descartar os piores cedo)
    "epochs": [5, 10, 20],
}

# 2. Configurar o KerasRegressor
nn_model = KerasRegressor(model=modelo_com_input, verbose=0)

## Atenção

- Ao juntar as features com o target, foram colocadas 10 simulações associadas ao mesmo target. 

- Ao usar a validação cruzada é preciso ter certeza de que as linhas associadas a um dado valor do target, caiam tanto na validação quanto no treino, evitando assim o vazamento do target. 

- Para isso utiliza-se o parâmetro `cv=gkf` no HalvingRandomSearch abaixo.

- A variável gkf usa a classe GroupKFold para levar em consideração a observação acima e evitar o vazamento do target.

In [ ]:
groups = y_train.to_numpy().ravel()

In [ ]:
groups

Abaixo, verifica-se se realmente não há vazamento de target usando-se o GrupoKFold

In [ ]:
# Inicialize o GroupKFold
gkf = GroupKFold(n_splits=5)

# Itere sobre os splits (apenas a primeira dobra é suficiente para checar)
for fold, (train_index, val_index) in enumerate(
    gkf.split(X_train, y_train, groups=groups)
):
    if fold == 0:  # Checar apenas o primeiro fold
        # 1. Obter os valores de 'r' (originais, não escalados) para Treino e Validação
        r_train_fold = y_train[train_index].to_numpy().ravel()
        r_val_fold = y_train[val_index].to_numpy().ravel()

        # 2. Encontrar os valores ÚNICOS de 'r' em cada conjunto
        r_unique_train = set(r_train_fold)
        r_unique_val = set(r_val_fold)

        # 3. Encontrar a Interseção (os valores vazados)
        vazamentos = r_unique_train.intersection(r_unique_val)

        print(f"--- Checagem do Fold {fold + 1} ---")
        print(f"Total de valores únicos de 'r' no Treino: {len(r_unique_train)}")
        print(f"Total de valores únicos de 'r' na Validação: {len(r_unique_val)}")
        print(f"Valores de 'r' vazando (Interseção): {len(vazamentos)}")

        if len(vazamentos) == 0:
            print(
                "✅ GroupKFold está funcionando corretamente: Nenhuma intersecção de valores de 'r'."
            )
        else:
            print(
                f"❌ ERRO GRAVE: {len(vazamentos)} valores de 'r' estão vazando! O GroupKFold falhou na divisão dos grupos."
            )
            print(f"Valores vazados (Primeiros 5): {list(vazamentos)[:5]}")

        break  # Parar após o primeiro fold

In [ ]:
# factor=2: Descarta metade dos candidatos a cada iteração.
# candidates: O número total de combinações que serão testadas na 1ª rodada (a maior).
hrs = HalvingRandomSearchCV(
    estimator=nn_model,
    param_distributions=param_distributions,
    factor=2,
    n_candidates=50,
    scoring="r2",
    random_state=42,
    cv=gkf,
    verbose=2,
    n_jobs=n_jobs,
)

print(
    "Iniciando Halving Random Search (Testando as melhores combinações eficientemente)..."
)

# 4. Executar a busca
# O HRS executa a busca e o retreinamento (refit=True)
hrs_result = hrs.fit(X_train_scaled, y_train_scaled, groups=groups)

In [ ]:
print("\n" + "=" * 50)
print("RESULTADOS FINAIS DO HALVING RANDOM SEARCH")
print("=" * 50)

print(f"Número de features usadas: {len(selected_feature_names)}")

# Melhor pontuação
print(f"Melhor R² Médio: {hrs_result.best_score_:.4f}")

# Melhor combinação de hiperparâmetros
print("Melhores Parâmetros:")
print(hrs_result.best_params_)

# Obter o melhor modelo treinado
best_nn_model = hrs_result.best_estimator_

# Salva o  modelo gerado
joblib.dump(best_nn_model, caminho_modelo)

# --- AVALIAÇÃO FINAL ---


# O reshape é necessário pois o KerasRegressor.predict() retorna 1D por padrão no Scikit-Learn Wrapper
y_pred_scaled = best_nn_model.predict(X_train_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)

# 4. Prepara o target verdadeiro para comparação. Garante que y_train seja convertido de um pandas Series para um Numpy array, que é o objeto que sai da inverse_transform acima
y_true = y_train.values.reshape(-1, 1)


# Cálculo das métricas no conjunto de treino
final_mse = mean_squared_error(y_true, y_pred_original)
final_rmse = root_mean_squared_error(y_true, y_pred_original)
final_r2 = r2_score(y_true, y_pred_original)
final_mae = mean_absolute_error(y_true, y_pred_original)


print("\nMétricas do Melhor Modelo (Avaliadas nos dados completos de Treino):")
print(f"  MSE (Erro Quadrático Médio): {final_mse:.8f} (Penaliza erros grandes)")
print(f"  RMSE (Erro Quadrático Médio): {final_rmse:.8f} (Penaliza erros grandes)")
print(f"  MAE (Erro Absoluto Médio):   {final_mae:.8f} (Unidade de 'r')")
print(f"  R2 (Ajuste):                 {final_r2:.4f} (Qualidade do ajuste)")
print("=" * 50)

## Observação

- Comparado com os resultados nos dados de treino do notebook 2, todas as métricas melhoraram.

## 4.1 - Visualizando os resultados

Abaixo, são mostradas duas visualizações:

- A distribuição dos valores preditos comparados com os reais,

- A distribuição dos resíduos.

In [ ]:
y_train.size

In [ ]:
y_train

In [ ]:
y_pred_original.size

In [ ]:
y_pred_original

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Suponha que você já tem:
# y_true_original: valores verdadeiros de 'r' (na escala original)
# y_pred_original: valores previstos de 'r' (na escala original)

# Crie a linha de identidade X=Y
min_val = min(y_train.min(), y_pred_original.min())
max_val = max(y_train.max(), y_pred_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_train, y_pred_original, alpha=0.6, s=20, label="Previsões")

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

plt.title(f"Previsões vs. Valores Reais de r (R² Final: {final_r2:.4f})", fontsize=14)
plt.xlabel("Valores Reais de r", fontsize=12)
plt.ylabel("Valores Previstos de r", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.gca().set_aspect(
    "equal", adjustable="box"
)  # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_true - y_pred_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor="black", alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Erro Zero")

plt.title("Distribuição dos Resíduos (Erros)", fontsize=14)
plt.xlabel("Resíduo (Real r - Previsto r)", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

In [ ]:
# --- PREPARAÇÃO ---
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_true_flat = y_true.ravel()
y_pred_flat = y_pred_original.ravel()

df_results = pd.DataFrame({"y_true": y_true_flat, "y_pred": y_pred_flat})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = df_results.groupby("y_true")["y_pred"].agg(["mean", "std"]).reset_index()

# Renomear as colunas para clareza
df_grouped.columns = ["y_true", "y_pred_mean", "y_pred_std"]


# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped["y_pred_std_safe"] = df_grouped["y_pred_std"].replace(0, 1e-6)
df_grouped["Standardized_Residual"] = (
    np.abs(df_grouped["y_true"] - df_grouped["y_pred_mean"])
    / df_grouped["y_pred_std_safe"]
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

## Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

Uma forma mais útil de visualizar os dados é portanto colapsar os 10 dados referentes à cada valor do target em seu valor médio e desvio padrão.

Além disso, plota-se gráfico de resíduos padronizados, que ajuda  visualizar o número de observações que se distanciam do valor real em mais de um desvio-padrão.



In [ ]:
# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 10),
    sharex=True,  # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={"hspace": 0.05},  # Reduz o espaço entre os subplots
    constrained_layout=True,
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped["y_true"],
    y=df_grouped["y_pred_mean"],
    yerr=df_grouped["y_pred_std"],
    fmt="o",
    capsize=4,
    alpha=0.7,
    label="Previsão Média ± 1 DP",
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

ax1.set_title(f"Análise de Previsão Agregada (R² Final: {final_r2:.4f})", fontsize=14)
ax1.set_ylabel("Valores Previstos Médios de r", fontsize=12)
ax1.grid(True, linestyle=":", alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped["y_true"],
    df_grouped["Standardized_Residual"],
    alpha=0.7,
    s=30,
    label="|Resíduo| / DP",
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, color="red", linestyle="-", linewidth=2, label="Limite de 1 Desvio Padrão"
)
ax2.axhline(y=2.0, color="orange", linestyle="--", linewidth=1, label="Limite de 2 DP")


ax2.set_xlabel("Valores Reais de r (Y_true)", fontsize=12)
ax2.set_ylabel("Resíduo Padronizado (|Y - Ŷ| / DP)", fontsize=12)
ax2.grid(True, linestyle=":", alpha=0.7)
ax2.legend()

plt.show()

## Obsevação

- Houve uma melhora nas métricas, mas fica evidente nas visualizações acima que houve uma piora nas previsões para valores muito baixo do target 'r'.

- O modelo tende a errar mais para valores de r próximos de 0. 

# 5 - Aplicando o modelo no conjunto de teste

In [ ]:
teste = pd.read_csv("../data/02_intermediate/teste_b_df.csv")

In [ ]:
teste

In [ ]:
X_test = teste.drop(columns=["r"])
y_test = teste["r"]

## 5.1 - Criando features extras

Todas as features criadas para treinar o modelo devem também ser criadas ao se aplicar os dados de teste. 

In [ ]:
X_test = utils.cria_feature_media_ponderada(X_test)

In [ ]:
X_test

## 5.2 - Selecionando features

In [ ]:
X_test = X_test[selected_feature_names]

## 5.3 - Normalizando os dados de teste 

In [ ]:
X_test_scaled = X_scaler.fit_transform(X_test)

# Escalonamento do Target (Y)
# Reformatar y_train para que o scaler funcione (de Series para 2D array/DataFrame)
y_test_2d = y_test.values.reshape(-1, 1)
y_test_scaled = y_scaler.fit_transform(y_test_2d)

In [ ]:
X_test_scaled

In [ ]:
X_test_scaled.max()

In [ ]:
X_test_scaled.min()

## Salvando valores normalizados

As features normalizadas serão necessárias no notebook de explicabilidade dos modelos, por isso salva-se abaixo:




In [ ]:
np.save(caminho_X_test_linear_features, X_train_scaled)

np.save(caminho_y_test_linear_features, y_train_scaled)

In [ ]:
# Obtem o melhor modelo do Halving Random Search
best_nn_model = hrs_result.best_estimator_

# A previsão é feita na escala Z-Score (escalada)
y_pred_test_scaled = best_nn_model.predict(X_test_scaled).reshape(-1, 1)

# Faz a transformação inversa para obter os valores em 'r':
y_pred_test_original = y_scaler.inverse_transform(y_pred_test_scaled)


print("Previsão no Conjunto de Teste concluída com sucesso.")

In [ ]:
X_test_scaled.max()

In [ ]:
y_pred_test_scaled.max()

In [ ]:
X_test_scaled.min()

In [ ]:
y_pred_test_scaled.min()

In [ ]:
y_pred_test_original.max()

In [ ]:
y_pred_test_original.min()

## 5. 4 - Visualizando métricas

In [ ]:
# Cálculo das 3 Métricas de Avaliação Finais:
mse_test = mean_squared_error(y_test_2d, y_pred_test_original)
rmse_test = root_mean_squared_error(y_test_2d, y_pred_test_original)
mae_test = mean_absolute_error(y_test_2d, y_pred_test_original)
r2_test = r2_score(y_test_2d, y_pred_test_original)

print("\n" + "=" * 50)
print("AVALIAÇÃO DE DESEMPENHO NO CONJUNTO DE TESTE")
print("=" * 50)
print(f"MSE (Erro Quadrático Médio): {mse_test:.8f}")
print(f"RMSE (Erro Absoluto Médio):   {rmse_test:.8f} (Erro médio na unidade de 'r')")
print(f"MAE (Erro Absoluto Médio):   {mae_test:.8f} (Erro médio na unidade de 'r')")
print(f"R2 (Ajuste):                 {r2_test:.4f}")
print("=" * 50)

In [ ]:
# Crie a linha de identidade X=Y
min_val = min(y_test_2d.min(), y_pred_test_original.min())
max_val = max(y_test_2d.max(), y_pred_test_original.max())
ideal_line = np.linspace(min_val, max_val, 100)

plt.figure(figsize=(8, 8))

# 1. Scatter Plot dos Resultados
plt.scatter(y_test_2d, y_pred_test_original, alpha=0.6, s=20, label="Previsões")

# 2. Linha de Identidade (Ajuste Perfeito)
plt.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

plt.title(f"Previsões vs. Valores Reais de r (R² Final: {r2_test:.4f})", fontsize=14)
plt.xlabel("Valores Reais de r", fontsize=12)
plt.ylabel("Valores Previstos de r", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.gca().set_aspect(
    "equal", adjustable="box"
)  # Garante que os eixos tenham a mesma escala
plt.show()

In [ ]:
# Calcule os resíduos (True - Predicted)
residuals = y_test_2d - y_pred_test_original

# Achata o array para 1D (o formato (N,))
residuals_1d = residuals.ravel()

plt.figure(figsize=(8, 6))

# Histograma dos resíduos
plt.hist(residuals_1d, bins=30, edgecolor="black", alpha=0.7)

# Linha vertical em zero (onde o centro do histograma deveria estar)
plt.axvline(x=0, color="red", linestyle="--", linewidth=2, label="Erro Zero")

plt.title("Distribuição dos Resíduos (Erros)", fontsize=14)
plt.xlabel("Resíduo (Real r - Previsto r)", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)
plt.show()

## 5.5 - Visualização com barra de erros

Os dados utilizados associam 10 valores diferentes das features a um mesmo valor do target. O objetivo é simular os ruídos presentes em observações astronômicas.

A visualização com barras de erros também é feita abaixo utilizando-se os dados de teste.

In [ ]:
# --- PREPARAÇÃO ---
# Criando um DataFrame para facilitar a agregação
# Garantindo que y_train e y_pred_original são 1D para o DataFrame
y_test_flat = y_test_2d.ravel()
y_pred_test_flat = y_pred_test_original.ravel()

df_results = pd.DataFrame({"y_test": y_test_flat, "y_test_pred": y_pred_test_flat})

# AGREGAR: Agrupar por 'y_true' e calcular Média e Desvio Padrão para 'y_pred'
# 'y_true' (a variável agrupada) será o índice do novo DataFrame
df_grouped = (
    df_results.groupby("y_test")["y_test_pred"].agg(["mean", "std"]).reset_index()
)

# Renomear as colunas para clareza
df_grouped.columns = ["y_test", "y_test_pred_mean", "y_test_pred_std"]

# Cálculo do Resíduo Padronizado (em módulos, conforme solicitado)
# Evite a divisão por zero, substituindo desvios padrão zero por um valor pequeno (ex: 1e-6)
df_grouped["y_test_pred_std_safe"] = df_grouped["y_test_pred_std"].replace(0, 1e-6)
df_grouped["Standardized_Residual"] = (
    np.abs(df_grouped["y_test"] - df_grouped["y_test_pred_mean"])
    / df_grouped["y_test_pred_std_safe"]
)

# Exibir o resultado da agregação (opcional)
print("Dados Agrupados (Exemplo):")
print(df_grouped.head())

In [ ]:
# --- VISUALIZAÇÃO COM GRÁFICO DE DUPLO EIXO ---

# Cria um espaço de figuras com 2 linhas e 1 coluna, compartilhando o eixo X
fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 10),
    sharex=True,  # Os dois gráficos compartilham o mesmo eixo X
    gridspec_kw={"hspace": 0.05},  # Reduz o espaço entre os subplots
    constrained_layout=True,
)

# ==========================================================
# GRÁFICO SUPERIOR: Previsão Média vs. Real (Com Barras de Erro)
# ==========================================================

# 1. Plotar os pontos agregados com Barras de Erro
ax1.errorbar(
    x=df_grouped["y_test"],
    y=df_grouped["y_test_pred_mean"],
    yerr=df_grouped["y_test_pred_std"],
    fmt="o",
    capsize=4,
    alpha=0.7,
    label="Previsão Média ± 1 DP",
)

# 2. Linha de Identidade (Ajuste Perfeito Y=X)
ax1.plot(
    ideal_line,
    ideal_line,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Ajuste Perfeito (Y=X)",
)

ax1.set_title(f"Análise de Previsão Agregada (R² Final: {final_r2:.4f})", fontsize=14)
ax1.set_ylabel("Valores Previstos Médios de r", fontsize=12)
ax1.grid(True, linestyle=":", alpha=0.7)
ax1.legend()

# ==========================================================
# GRÁFICO INFERIOR: Resíduos Padronizados
# ==========================================================

# 1. Scatter Plot dos Resíduos Padronizados
ax2.scatter(
    df_grouped["y_test"],
    df_grouped["Standardized_Residual"],
    alpha=0.7,
    s=30,
    label="|Resíduo| / DP",
)

# 2. Linha de Referência Crítica (Z-Score = 1.0)
ax2.axhline(
    y=1.0, color="red", linestyle="-", linewidth=2, label="Limite de 1 Desvio Padrão"
)
ax2.axhline(y=2.0, color="orange", linestyle="--", linewidth=1, label="Limite de 2 DP")


ax2.set_xlabel("Valores Reais de r(Y_true)", fontsize=12)
ax2.set_ylabel("Resíduo Padronizado (|Y - Ŷ| / DP)", fontsize=12)
ax2.grid(True, linestyle=":", alpha=0.7)
ax2.legend()

plt.show()

# 6 - Sugestões para trabalhos futuros

- Fazer uma seleção de features utilizando-se um modelo linear ao invés do método estatístico utilizado pela classe SelectKBest.

- Testar o resultado do modelo eliminando menos features.

- Criar novas features com os dados de entrada.

- Aumentar a rede de parâmetros da validação cruzada.